In [1]:
%load_ext autoreload
%autoreload 2

In [5]:
#Benchmark criteria

# You may need more packages to test this benchmarking scripts. 
# These packages were NOT included in the package requirements because they are not necessary for actually using scplode. 



- Amount of memory it used. 
- Speed of access at random locations.
- Speed of access at contiguous locations. 
- Vertical access. 
- Benchmark against h5py
- Benchmark against CSR encoded data and raw data. 
    

#Keeping it simple:
    - Rarely do we need to write, and if so, best to not overwrite the raw anyways. 
    - So skipping that. 

        
# How fast is state with it?

SyntaxError: invalid syntax (111194518.py, line 8)

In [10]:
#Imports
import time
import psutil
import os
import gc
import numpy as np
import pandas as pd
import anndata as ad
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from memory_profiler import memory_usage
import tracemalloc

# Import your Scplode module
from scplode import read_h5ad, Scplode

# Set up plotting style
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)

In [7]:
#Helper functions for benchmarking

def get_process_memory():
    """Get current process memory usage in MB"""
    process = psutil.Process(os.getpid())
    return process.memory_info().rss / 1024 / 1024

def time_function(func, *args, **kwargs):
    """Time a function execution"""
    start = time.perf_counter()
    result = func(*args, **kwargs)
    end = time.perf_counter()
    return end - start, result

def measure_memory_peak(func, *args, **kwargs):
    """Measure peak memory usage of a function"""
    mem_usage = memory_usage((func, args, kwargs), interval=0.1, timeout=None)
    return max(mem_usage) - min(mem_usage)

class BenchmarkResults:
    """Store and visualize benchmark results"""
    def __init__(self):
        self.results = []
    
    def add_result(self, method, operation, time_sec, memory_mb, n_cells=None, n_genes=None):
        self.results.append({
            'method': method,
            'operation': operation,
            'time_sec': time_sec,
            'memory_mb': memory_mb,
            'n_cells': n_cells,
            'n_genes': n_genes
        })
    
    def to_dataframe(self):
        return pd.DataFrame(self.results)
    
    def plot_results(self):
        df = self.to_dataframe()
        
        fig, axes = plt.subplots(1, 2, figsize=(14, 6))
        
        # Time comparison
        pivot_time = df.pivot(index='operation', columns='method', values='time_sec')
        pivot_time.plot(kind='bar', ax=axes[0])
        axes[0].set_title('Access Time Comparison')
        axes[0].set_ylabel('Time (seconds)')
        axes[0].set_xlabel('Operation')
        axes[0].legend(title='Method')
        axes[0].set_yscale('log')
        
        # Memory comparison
        pivot_memory = df.pivot(index='operation', columns='method', values='memory_mb')
        pivot_memory.plot(kind='bar', ax=axes[1])
        axes[1].set_title('Memory Usage Comparison')
        axes[1].set_ylabel('Memory (MB)')
        axes[1].set_xlabel('Operation')
        axes[1].legend(title='Method')
        
        plt.tight_layout()
        return fig


In [30]:
## Path to your h5ad file
h5ad_path = Path("/Volumes/T7/vcc_pp/data/raw/competition_support_set/competition_train.h5")  # Update with your actual file path


In [31]:

benchmark = BenchmarkResults()

# Get file info
file_size_mb = h5ad_path.stat().st_size / 1024 / 1024
print(f"H5AD file size: {file_size_mb:.2f} MB")

# Test 1: Load with standard AnnData (in-memory)
print("\n--- Loading with AnnData (in-memory) ---")
gc.collect()
initial_mem = get_process_memory()
time_taken, adata_mem = time_function(ad.read_h5ad, h5ad_path)
memory_used = get_process_memory() - initial_mem
print(f"Time: {time_taken:.3f}s, Memory: {memory_used:.2f} MB")
benchmark.add_result('AnnData (memory)', 'initial_load', time_taken, memory_used)

# Test 2: Load with AnnData (backed mode)
print("\n--- Loading with AnnData (backed) ---")
del adata_mem
gc.collect()
initial_mem = get_process_memory()
time_taken, adata_backed = time_function(ad.read_h5ad, h5ad_path, backed='r')
memory_used = get_process_memory() - initial_mem
print(f"Time: {time_taken:.3f}s, Memory: {memory_used:.2f} MB")
benchmark.add_result('AnnData (backed)', 'initial_load', time_taken, memory_used)

# Test 3: Load with Scplode (first time - creates index)
print("\n--- Loading with Scplode (creating index) ---")
# Clean up any existing index files
scplode_temp = Scplode(h5ad_path)
scplode_temp.delete_index()
del scplode_temp
gc.collect()

initial_mem = get_process_memory()
time_taken, scdata = time_function(read_h5ad, h5ad_path)
memory_used = get_process_memory() - initial_mem
print(f"Time: {time_taken:.3f}s, Memory: {memory_used:.2f} MB")
benchmark.add_result('Scplode (first)', 'initial_load', time_taken, memory_used)

# Test 4: Load with Scplode (subsequent times - uses index)
print("\n--- Loading with Scplode (using existing index) ---")
del scdata
gc.collect()
initial_mem = get_process_memory()
time_taken, scdata = time_function(read_h5ad, h5ad_path)
memory_used = get_process_memory() - initial_mem
print(f"Time: {time_taken:.3f}s, Memory: {memory_used:.2f} MB")
benchmark.add_result('Scplode (cached)', 'initial_load', time_taken, memory_used)

# Keep one instance of each for further testing
adata_mem = ad.read_h5ad(h5ad_path)
n_cells, n_genes = adata_mem.shape
print(f"\nDataset shape: {n_cells} cells x {n_genes} genes")
    


H5AD file size: 14764.88 MB

--- Loading with AnnData (in-memory) ---
Time: 30.177s, Memory: 585.91 MB

--- Loading with AnnData (backed) ---


[WARNING] File not found, skipping: /Volumes/T7/vcc_pp/data/raw/competition_support_set/competition_train.obs
[WARNING] File not found, skipping: /Volumes/T7/vcc_pp/data/raw/competition_support_set/competition_train.var
[WARNING] File not found, skipping: /Volumes/T7/vcc_pp/data/raw/competition_support_set/competition_train.dat
[WARNING] File not found, skipping: /Volumes/T7/vcc_pp/data/raw/competition_support_set/competition_train.mtime
[INFO] Creating index
[INFO] Creating index: reading adata file


Time: 0.133s, Memory: 32.89 MB

--- Loading with Scplode (creating index) ---


[INFO] Creating index: writing mmap dat file


  0%|          | 0/222 [00:00<?, ?it/s]

[INFO] Creating index: packing obs
[INFO] Creating index: packing var
[INFO] Loading index: obs
[INFO] Loading index: var
[INFO] Loading index: dat (implicitly)
[INFO] Loading index: obs
[INFO] Loading index: var
[INFO] Loading index: dat (implicitly)


Time: 91.004s, Memory: 403.40 MB

--- Loading with Scplode (using existing index) ---
Time: 0.035s, Memory: 9.54 MB

Dataset shape: 221273 cells x 18080 genes


In [32]:
# Generate random indices for testing
np.random.seed(42)
test_sizes = [10, 100, 1000, 5000]
cell_indices = {
    size: np.random.choice(adata_backed.obs.index, size, replace=False)  # Changed from adata_mem
    for size in test_sizes
}

print("\n=== Random Cell Access Benchmark ===")
for size in test_sizes:
    indices = cell_indices[size]
    print(f"\n--- Accessing {size} random cells ---")
    
    # AnnData backed
    gc.collect()
    initial_mem = get_process_memory()
    time_taken, _ = time_function(lambda: adata_backed[indices].to_memory())  # Fixed syntax
    memory_used = get_process_memory() - initial_mem
    print(f"AnnData (backed): {time_taken:.4f}s, {memory_used:.2f} MB")
    benchmark.add_result('AnnData (backed)', f'access_{size}_cells', time_taken, memory_used)
    
    # Scplode
    gc.collect()
    initial_mem = get_process_memory()
    time_taken, _ = time_function(scdata.get_adata, indices)  # Fixed syntax
    memory_used = get_process_memory() - initial_mem
    print(f"Scplode: {time_taken:.4f}s, {memory_used:.2f} MB")
    benchmark.add_result('Scplode', f'access_{size}_cells', time_taken, memory_used)


=== Random Cell Access Benchmark ===

--- Accessing 10 random cells ---
AnnData (backed): 0.0584s, -46.04 MB
Scplode: 0.0670s, -5.48 MB

--- Accessing 100 random cells ---
AnnData (backed): 0.2975s, 10.69 MB
Scplode: 0.1751s, 6.36 MB

--- Accessing 1000 random cells ---
AnnData (backed): 2.8688s, -519.51 MB
Scplode: 2.3473s, -107.51 MB

--- Accessing 5000 random cells ---
AnnData (backed): 14.3035s, -81.88 MB
Scplode: 11.5378s, -100.04 MB


In [34]:
import time
import gc
import psutil
import numpy as np

process = psutil.Process()

def get_rss_mb():
    """Return current RSS memory in MB."""
    return process.memory_info().rss / 1024**2

def time_function(func, *args, repeat=3):
    """Time a function and return average time and memory."""
    times = []
    mems = []
    for _ in range(repeat):
        gc.collect()
        start_mem = get_rss_mb()
        start_time = time.time()
        result = func(*args)
        elapsed = time.time() - start_time
        mem_diff = get_rss_mb() - start_mem
        times.append(elapsed)
        mems.append(mem_diff)
    return np.mean(times), np.mean(mems), result

def memmap_mean(memmap_path, row_indices, n_vars, chunk_size=1000):
    """Compute mean of selected rows in memmap without loading all at once."""
    data = np.memmap(memmap_path, dtype="float32", mode="r").reshape(-1, n_vars)
    mean_accum = np.zeros(n_vars, dtype=np.float64)
    count = 0
    for start in range(0, len(row_indices), chunk_size):
        chunk_idx = row_indices[start:start+chunk_size]
        mean_accum += data[chunk_idx, :].sum(axis=0)
        count += len(chunk_idx)
    return mean_accum / count

# Random indices for testing
np.random.seed(42)
test_sizes = [10, 100, 1000, 5000]
cell_indices = {
    size: np.random.choice(adata_backed.obs.index, size, replace=False)
    for size in test_sizes
}

print("\n=== Random Cell Mean Benchmark ===")
for size in test_sizes:
    indices = cell_indices[size]
    print(f"\n--- Accessing {size} random cells ---")

    # AnnData (backed) mean
    def adata_mean(idx):
        return adata_backed[idx].X.mean(axis=0)

    time_taken, mem_used, _ = time_function(adata_mean, indices)
    print(f"AnnData (backed): {time_taken:.4f}s, {mem_used:.2f} MB")

    # Scplode mean (chunked)
    row_positions = scdata.obs.index.get_indexer(indices)
    def scplode_mean(idx_pos):
        return memmap_mean(scdata.dat_path, idx_pos, scdata.n_vars)

    time_taken, mem_used, _ = time_function(scplode_mean, row_positions)
    print(f"Scplode (chunked): {time_taken:.4f}s, {mem_used:.2f} MB")


=== Random Cell Mean Benchmark ===

--- Accessing 10 random cells ---
AnnData (backed): 0.0356s, 7.20 MB
Scplode (chunked): 0.0096s, 0.18 MB

--- Accessing 100 random cells ---
AnnData (backed): 0.2007s, 19.13 MB
Scplode (chunked): 0.1189s, 3.55 MB

--- Accessing 1000 random cells ---
AnnData (backed): 1.8573s, 103.60 MB
Scplode (chunked): 0.6243s, 21.46 MB

--- Accessing 5000 random cells ---
AnnData (backed): 21.2922s, 268.01 MB
Scplode (chunked): 4.7727s, -9.14 MB


In [102]:
import time
import tracemalloc
import numpy as np


# Utility to time function and measure peak memory
def time_and_memory(repeat=3):
    times = []
    peaks = []
    results = list()
    for _ in range(repeat):
        tracemalloc.start()
        start_time = time.time()
        indices = random.sample(list(adata_backed.obs.index), 5000)
        row_positions = scdata.obs.index.get_indexer(indices)
        result = adata_backed[indices].X.var(axis=0)
        elapsed = time.time() - start_time
        current, peak = tracemalloc.get_traced_memory()
        tracemalloc.stop()
        times.append(elapsed)
        peaks.append(peak / 1024**2)  # MB
        results.append(result)
    return np.mean(times), np.mean(peaks), results[0]

dt_adata, mem_adata, _ = time_and_memory()
print(f"AnnData (backed): {dt_adata:.4f}s, peak memory: {mem_adata:.2f} MB")



AttributeError: 'csr_matrix' object has no attribute 'var'

In [103]:
import time
import tracemalloc
import numpy as np


# Utility to time function and measure peak memory
def time_and_memory(repeat=3):
    times = []
    peaks = []
    results = list()
    for _ in range(repeat):
        tracemalloc.start()
        start_time = time.time()
        indices = random.sample(list(adata_backed.obs.index), 5000)
        row_positions = scdata.obs.index.get_indexer(indices)
        result = scdata[indices].X.var(axis=0)
        elapsed = time.time() - start_time
        current, peak = tracemalloc.get_traced_memory()
        tracemalloc.stop()
        times.append(elapsed)
        peaks.append(peak / 1024**2)  # MB
        results.append(result)
    return np.mean(times), np.mean(peaks), results[0]

dt_adata, mem_adata, _ = time_and_memory()
print(f"AnnData (backed): {dt_adata:.4f}s, peak memory: {mem_adata:.2f} MB")


AnnData (backed): 12.1925s, peak memory: 1146.57 MB


In [99]:
indices = random.sample(list(adata_backed.obs.index), 5000)
scdata.obs.index.get_indexer[indices]

TypeError: 'method' object is not subscriptable